In [3]:
%run /home/dbaciur/NTU/NTU/notes/Constants.ipynb

Constants loaded


In this file we want to check all existing subjects

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torchvision as tv

## Import annotations

In [6]:
annotations_df = pd.read_csv(C.PROCESSED_ANNOTATIONS_PATH)

/home/bjastrze/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
annotations_df.head(1)

,Id,Unnamed: 0,Behavior,Behavior type,Behavioral category,Comment start,Comment stop,Duration (s),FPS,Media file,...,Start (s),Stop (s),Subject,Total length,lecture_id,research_assistant_id,subject_id,BehaviorId,Video,Time
0,0,0,eye contact,STATE,Encouraging learner engagement,0.0,0,2841.008,29.83,C:/Users/ACER/Downloads/PH1011-PHYSICS_2015092...,...,29.355,2870.363,No focal subject,7197.58,J1S1C11L1,RA13,J1S1C11,0,PH1011-PHYSICS_20150922,1449.859


### Analyze available courses / subjects

In [8]:
subjects = annotations_df[C.A_SUBJECT_ID].unique()
subjects

array(['J1S1C11', 'J1S1C12', 'J1S1C13', 'J1S1C14', 'J1S1C5', 'J1S2C15',
       'J1S2C4', 'J2S1C10', 'J2S1C13', 'J2S1C17', 'J2S2C3', 'J3S1C11',
       'J3S1C17', 'J3S1C1', 'J3S1C2', 'J3S2C16', 'J3S2C18', 'J3S2C3',
       'J3S2C7', 'J3S2C8', 'J3S2C9', 'J4S1C2', 'J4S1C6'], dtype=object)

### Group all videos by subject id

In [9]:
videos = annotations_df[C.A_VIDEO].unique()
subjects_dict = {}

for video in videos:
    
    videos_df = annotations_df[annotations_df[C.A_VIDEO] == video]
    subjects = videos_df[C.A_SUBJECT_ID].unique()
    
    if len(subjects) > 1:
        print(subjects)
    subject = subjects[0]
    
    if subjects_dict.get(subject) is None:
        subjects_dict[subject] = []
    
    subjects_dict[subject].append(video)

subjects_dict

{'J1S1C11': ['PH1011-PHYSICS_20150922',
  'PH1011-PHYSICS_20151006',
  'PH1011-PHYSICS_20151013',
  'PH1011-PHYSICS_20151020'],
 'J1S1C12': ['15S1-PH1012-LEC_20150812'],
 'J1S1C13': ['PH1104-MECHANICS_20150817_PC1',
  '15S1-PH1104-LEC_20150820_PC1',
  'PH1104-MECHANICS_20150824_PC1',
  'PH1104-MECHANICS_20150827_PC1',
  'PH1104-MECHANICS_20150831_PC1'],
 'J1S1C14': ['15S1-PH1105-LEC_20150817',
  '15S1-PH1105-LEC_20150821',
  '15S1-PH1105-LEC_20150824',
  '15S1-PH1105-LEC_20150828'],
 'J1S1C5': ['15S1-MH1800-LEC_20150817',
  '15S1-MH1800-LEC_20150824',
  '15S1-MH1800-LEC_20150831'],
 'J1S2C15': ['15S2-PH1106-LEC_20160113',
  '15S2-PH1106-LEC_20160120',
  '15S2-PH1106-LEC_20160127',
  '15S2-PH1106-LEC_20160203',
  '15S2-PH1106-LEC_20160210'],
 'J1S2C4': ['15S2-MH1402-LEC_20160115',
  '15S2-MH1402-LEC_20160122',
  '15S2-MH1402-LEC_20160129',
  '15S2-MH1402-LEC_20160205',
  '15S2-MH1402-LEC_20160212'],
 'J2S1C10': ['16S1-MH1812-LEC_20160808',
  '16S1-MH1812-LEC_20160811',
  '16S1-MH1812-LE

In [10]:
len(subjects_dict)

23

In [14]:
## This is just apretty print of previous output
subjects_dict = {
   "J1S1C11":[
      "PH1011-PHYSICS_20150922",
      "PH1011-PHYSICS_20151006",
      "PH1011-PHYSICS_20151013",
      "PH1011-PHYSICS_20151020"
   ],
   "J1S1C12":[
      "15S1-PH1012-LEC_20150812"
   ],
   "J1S1C13":[
      "PH1104-MECHANICS_20150817_PC1",
      "15S1-PH1104-LEC_20150820_PC1",
      "PH1104-MECHANICS_20150824_PC1",
      "PH1104-MECHANICS_20150827_PC1",
      "PH1104-MECHANICS_20150831_PC1"
   ],
   "J1S1C14":[
      "15S1-PH1105-LEC_20150817",
      "15S1-PH1105-LEC_20150821",
      "15S1-PH1105-LEC_20150824",
      "15S1-PH1105-LEC_20150828"
   ],
   "J1S1C5":[
      "15S1-MH1800-LEC_20150817",
      "15S1-MH1800-LEC_20150824",
      "15S1-MH1800-LEC_20150831"
   ],
   "J1S2C15":[
      "15S2-PH1106-LEC_20160113",
      "15S2-PH1106-LEC_20160120",
      "15S2-PH1106-LEC_20160127",
      "15S2-PH1106-LEC_20160203",
      "15S2-PH1106-LEC_20160210"
   ],
   "J1S2C4":[
      "15S2-MH1402-LEC_20160115",
      "15S2-MH1402-LEC_20160122",
      "15S2-MH1402-LEC_20160129",
      "15S2-MH1402-LEC_20160205",
      "15S2-MH1402-LEC_20160212"
   ],
   "J2S1C10":[
      "16S1-MH1812-LEC_20160808",
      "16S1-MH1812-LEC_20160811",
      "16S1-MH1812-LEC_20160815",
      "16S1-MH1812-LEC_20160818",
      "16S1-MH1812-LEC_20160822"
   ],
   "J2S1C13":[
      "16S1-PH1104-LEC_20161024",
      "16S1-PH1104-LEC_20161027",
      "16S1-PH1104-LEC_20161031",
      "16S1-PH1104-LEC_20161103",
      "16S1-PH1104-LEC_20161107",
      "16S1-PH1104-LEC_20161110"
   ],
   "J2S1C17":[
      "16S1-PH1801-LEC_20161013",
      "16S1-PH1801-LEC_20161020",
      "16S1-PH1801-LEC_20161027",
      "16S1-PH1801-LEC_20161103",
      "16S1-PH1801-LEC_20161110"
   ],
   "J2S2C3":[
      "16S2-MH1301-LEC_20170109",
      "16S2-MH1301-LEC_20170116",
      "16S2-MH1301-LEC_20170123",
      "16S2-MH1301-LEC_20170206",
      "16S2-MH1301-LEC_20170213"
   ],
   "J3S1C11":[
      "17S1-PH1011-LEC_20171010",
      "17S1-PH1011-LEC_20171017",
      "17S1-PH1011-LEC_20171024",
      "17S1-PH1011-LEC_20171031",
      "17S1-PH1011-LEC_20170912"
   ],
   "J3S1C17":[
      "17S1-PH1801-LEC_20170817",
      "17S1-PH1801-LEC_20170824",
      "17S1-PH1801-LEC_20170831",
      "17S1-PH1801-LEC_20170907",
      "17S1-PH1801-LEC_20170914",
      "17S1-PH1801-LEC_20170921"
   ],
   "J3S1C1":[
      "17S1-CM1021-LEC_20170823",
      "17S1-CM1021-LEC_20170906"
   ],
   "J3S1C2":[
      "17S1-MH1100-LEC_20170817",
      "17S1-MH1100-LEC_20171027",
      "17S1-MH1100-LEC_20170818",
      "17S1-MH1100-LEC_20170824",
      "17S1-MH1100-LEC_20170825",
      "17S1-MH1100-LEC_20170831",
      "17S1-MH1100-LEC_20171012",
      "17S1-MH1100-LEC_20171019",
      "17S1-MH1100-LEC_20171020",
      "17S1-MH1100-LEC_20171026"
   ],
   "J3S2C16":[
      "17S2-PH1107-CY1307-LEC_20180116",
      "17S2-PH1107-CY1307-LEC_20180123",
      "17S2-PH1107-CY1307-LEC_20180130",
      "17S2-PH1107-CY1307-LEC_20180206",
      "17S2-PH1107-CY1307-LEC_20180220"
   ],
   "J3S2C18":[
      "17S2-PH1802-LEC_20180316",
      "17S2-PH1802-LEC_20180324",
      "17S2-PH1802-LEC_20180406",
      "17S2-PH1802-LEC_20180413",
      "17S2-PH1802-LEC_20180420"
   ],
   "J3S2C3":[
      "17S2-MH1301-LEC_20180115",
      "17S2-MH1301-LEC_20180122",
      "17S2-MH1301-LEC_20180129",
      "17S2-MH1301-LEC_20180205",
      "17S2-MH1301-LEC_20180212",
      "17S2-MH1301-LEC_20180219"
   ],
   "J3S2C7":[
      "17S2-MH1803-LEC_20180116",
      "17S2-MH1803-LEC_20180117",
      "17S2-MH1803-LEC_20180123",
      "17S2-MH1803-LEC_20180124",
      "17S2-MH1803-LEC_20180130",
      "17S2-MH1803-LEC_20180131"
   ],
   "J3S2C8":[
      "17S2-MH1804-LEC_20180315",
      "17S2-MH1804-LEC_20180322",
      "17S2-MH1804-LEC_20180329",
      "17S2-MH1804-LEC_20180412",
      "17S2-MH1804-LEC_20180419"
   ],
   "J3S2C9":[
      "17S2-MH1811-LEC_20180205",
      "17S2-MH1811-LEC_20180207",
      "17S2-MH1811-LEC_20180212",
      "17S2-MH1811-LEC_20180214",
      "17S2-MH1811-LEC_20180219",
      "17S2-MH1811-LEC_20180221"
   ],
   "J4S1C2":[
      "18S1-MH1100-LEC_20180831",
      "18S1-MH1100-LEC_20181025",
      "18S1-MH1100-LEC_20181026",
      "18S1-MH1100-LEC_20180906",
      "18S1-MH1100-LEC_20180907",
      "18S1-MH1100-LEC_20180913",
      "18S1-MH1100-LEC_20180914",
      "18S1-MH1100-LEC_20181011",
      "18S1-MH1100-LEC_20181012",
      "18S1-MH1100-LEC_20181018",
      "18S1-MH1100-LEC_20181019"
   ],
   "J4S1C6":[
      "18S1-MH1802-LEC_20180820",
      "18S1-MH1802-LEC_20180827",
      "18S1-MH1802-LEC_20180829"
   ]
}

### Subjects to csv

In [15]:
lines = []
for subject, videos in subjects_dict.items():
    for video in videos:
        line = f"{subject},{video}"
        lines.append(line)
for line in lines:
    print(line)

J1S1C11,PH1011-PHYSICS_20150922
J1S1C11,PH1011-PHYSICS_20151006
J1S1C11,PH1011-PHYSICS_20151013
J1S1C11,PH1011-PHYSICS_20151020
J1S1C12,15S1-PH1012-LEC_20150812
J1S1C13,PH1104-MECHANICS_20150817_PC1
J1S1C13,15S1-PH1104-LEC_20150820_PC1
J1S1C13,PH1104-MECHANICS_20150824_PC1
J1S1C13,PH1104-MECHANICS_20150827_PC1
J1S1C13,PH1104-MECHANICS_20150831_PC1
J1S1C14,15S1-PH1105-LEC_20150817
J1S1C14,15S1-PH1105-LEC_20150821
J1S1C14,15S1-PH1105-LEC_20150824
J1S1C14,15S1-PH1105-LEC_20150828
J1S1C5,15S1-MH1800-LEC_20150817
J1S1C5,15S1-MH1800-LEC_20150824
J1S1C5,15S1-MH1800-LEC_20150831
J1S2C15,15S2-PH1106-LEC_20160113
J1S2C15,15S2-PH1106-LEC_20160120
J1S2C15,15S2-PH1106-LEC_20160127
J1S2C15,15S2-PH1106-LEC_20160203
J1S2C15,15S2-PH1106-LEC_20160210
J1S2C4,15S2-MH1402-LEC_20160115
J1S2C4,15S2-MH1402-LEC_20160122
J1S2C4,15S2-MH1402-LEC_20160129
J1S2C4,15S2-MH1402-LEC_20160205
J1S2C4,15S2-MH1402-LEC_20160212
J2S1C10,16S1-MH1812-LEC_20160808
J2S1C10,16S1-MH1812-LEC_20160811
J2S1C10,16S1-MH1812-LEC_2016081